# Using the library

## Basic usage

In [1]:
from csm_mlx.loaders import CSM
import time

load_start_time = time.time()
model = CSM()
load_end_time = time.time()

/Users/ritsuko/projects/ai/audio/csm_mlx/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 114688.00it/s]


This generates with a random speaker:

In [3]:
from IPython.display import Audio

text = "Hello, my name is John, and I like bread."
pcm = model(text, 0, temp=1, backbone_min_p=0.05, use_last_gens=True)

Audio(pcm, rate=model.sampling_rate)

186.083078ms prompt processing: 17 tokens (177.340144 tokens/s)


33it [00:02, 12.87it/s]

Generated in 2.75s (11.62 tokens/s, 86.02ms/token), 0.93x realtime


Optional: save audio procedurally

In [ ]:
from csm_mlx.io.wav import pcm_to_wav_bytes

with open("out.wav", "wb") as f:
    f.write(pcm_to_wav_bytes(pcm))

## Voice cloning

Get your reference audio. WAV files only for now, sorry.

In [2]:
from csm_mlx.loaders.csm import Segment
import soundfile as sf
from scipy.signal import resample
import numpy as np

def load_wav(path: str) -> np.ndarray:
    data, sr = sf.read(path)
    new_sample_rate = model.sampling_rate

    num_samples = int(len(data) * new_sample_rate / sr)

    # Resample using FFT-based method
    return resample(data, num_samples)

# Substitute with your segments as desired
audio = Segment(
    speaker=0,
    text="When I heard the release demo, I was shocked, angered, and in disbelief that Mr. Altman would pursue a voice that sounded so eerily similar to mine that my closest friends and news outlets could not tell the difference.",
    audio=load_wav("./tests/sky.wav")
)
context = [audio]

In [ ]:
from IPython.display import Audio

text = "This is a test of tet to speech."
pcm = model(text, 0, context=context, temp=1, backbone_min_p=0.1)

Audio(pcm, rate=model.sampling_rate)


952.437878ms prompt processing: 258 tokens (34.647929 tokens/s)


134it [00:10, 13.16it/s]


Generated in 11.14s (11.94 tokens/s, 83.77ms/token), 0.96x realtime


In [1]:
text = "This is a test of caching the previous mimi generations."
pcm = model(text, 0, use_last_gens=True, temp=0.9)

Audio(pcm, rate=model.sampling_rate)

NameError: name 'model' is not defined

## Streaming

In [ ]:
from IPython.display import Audio
import numpy as np

# TODO block anyway, just testing correctness
frames = [frame for frame in model.stream("Hello world, this is a test of streaming generation", context=context, speaker_id=0, use_last_gens=True, temp=0.8)]

Audio(np.array(frames).flatten(), rate=model.sampling_rate)